# 01 — Data Preparation (TMX → parallel text → train/dev/test)

**Purpose:**
 Prepare clean, aligned parallel data in the exact tab-separated format expected by NLLB and Hugging Face Datasets.

**Inputs:**

- `data/raw/source/source.txt` → Tagalog sentences.
- `data/raw/target/target.txt` → Cebuano translations.

**Process:**

- Filters short/long lines (for quality).
- Shuffles with the fixed seed for reproducibility.
- Splits into 80/10/10 train/dev/test.
- Writes to `data/processed/` as:
  - `train.tsv`
  - `dev.tsv`
  - `test.tsv`

**Outputs:**

- Aligned pairs of sentences for training and evaluation.
- Data sanity check printed via `df.sample()` (so you can visually confirm alignment).

In [1]:
import os, pathlib, sys
root = pathlib.Path(".")
print("Working directory:", root.resolve())
print("\nExpected files:")
print(" - src/prepare/convert_tmx.py")
print(" - src/prepare/make_splits.py")
print(" - data/raw/ceb-tl.tmx")

Working directory: D:\OneDrive\Documents\My Learning Resource\University Courses\DLSU\2025-26\T1\CSC715M\assignments\mc02\notebooks

Expected files:
 - src/prepare/convert_tmx.py
 - src/prepare/make_splits.py
 - data/raw/ceb-tl.tmx


## A) Convert TMX → `source.txt` & `target.txt`

This uses `lxml` with `recover=True` to tolerate broken XML.

In [5]:
!python ../src/prepare/convert_tmx.py --tmx ../data/raw/ceb-tl.tmx --src_lang tl --tgt_lang ceb --out_dir ../data/raw

Extracted 30,919 aligned pairs
Wrote:
  ..\data\raw\source\source.txt
  ..\data\raw\target\target.txt


## B) Clean & split into train/dev/test

In [6]:
!python ../src/prepare/make_splits.py --src ../data/raw/source/source.txt --tgt ../data/raw/target/target.txt --out ../data/processed

Wrote  24734 pairs → ..\data\processed\train.tsv
Wrote   3091 pairs → ..\data\processed\dev.tsv
Wrote   3093 pairs → ..\data\processed\test.tsv

Total usable pairs: 30,918


## C) Peek at the splits + quick QC


In [7]:
import pandas as pd, pathlib

pp = pathlib.Path("../data/processed")
for name in ["train.tsv", "dev.tsv", "test.tsv"]:
    df = pd.read_csv(pp / name, sep="\t", header=None, names=["src","tgt"])
    print(f"{name}: {len(df):,} pairs")
    display(df.sample(min(5, len(df))))

# Quick quality checks
train = pd.read_csv(pp/"train.tsv", sep="\t", header=None, names=["src","tgt"])
avg_src_len = train["src"].str.split().map(len).mean()
avg_tgt_len = train["tgt"].str.split().map(len).mean()
ratio = (train["src"].str.len() / train["tgt"].str.len()).clip(upper=10).mean()

print(f"\nAverage token length — SRC: {avg_src_len:.1f}, TGT: {avg_tgt_len:.1f}")
print(f"Average char length ratio (src/tgt, clipped): {ratio:.2f}")


train.tsv: 22,851 pairs


,src,tgt
20205,Ang mga lakad ng tao ay sa Panginoon; paano ng...,Ang mga pagpanlihok sa usa ka tawo iya ni Jeho...
2368,"At ako'y nakipagtalo sa kanila, at sinumpa ko ...","Ug ako nakiglalis kanila, ug nagtunglo kanila,..."
7092,"At kasama nila si Heman at si Jeduthun, at ang...","Ug uban kanila si Heman ug si Jeduthun, ug ang..."
22744,At sangpung bataan na tagadala ng sandata ni J...,Ug ang napulo ka mga batan-ong lalake nga nagd...
524,Ako'y nailabas ng wala pang mga kalaliman; nan...,"Sa diha nga wala pay kahiladman, ako nahimugso..."


dev.tsv: 2,930 pairs


,src,tgt
102,"Ano, wala baga kayong mga bahay na inyong maka...",Unsa? Wala ba kamoy mga balay diin adto kamo m...
1964,Bakit tinanggap ako ng mga tuhod? O bakit ng m...,Ngano nga ang mga tuhod mingdawat kanako? Kun ...
978,Kayo'y huwag gagawa ng anomang gawa: siyang pa...,Kamo dili magbuhat ug bisan unsa nga bulohaton...
1608,"At wala pa sa lupang kahoy sa parang, at wala ...",Ug wala pay tanum sa kapatagan nga diha sa iba...
89,At kung ang angkan ng Egipto ay hindi umahon a...,"Ug kong ang banay sa Egipto dili motungas, ug ..."


test.tsv: 2,750 pairs


,src,tgt
985,Masusumpungan ng iyong kamay ang lahat ng iyon...,Hipalgan sa imong kamot ang tanan nimong mga k...
352,"Ito nga ang sinasabi ko, at sinasaksihan sa Pa...",Busa magaingon ako ug magapamatuod niini tungo...
1678,"Kung gumawa ng kasamaan sa aking paningin, na ...",Kong buhaton nila kadtong dautan sa akong pagt...
841,Na hindi man kami nangakinig sa iyong mga ling...,Ni managpatalinghug kami sa imong mga alagad n...
45,"Mga anak, magsitalima kayo sa inyong magulang ...","Mga anak, magmasinugtanon kamo sa inyong mga g..."



Average token length — SRC: 26.6, TGT: 32.3
Average char length ratio (src/tgt, clipped): 0.97
